In [ ]:
import pandas
import arcgis

# read the 311 CSV
df_311 = pandas.read_csv("./311_cases.csv")

# drop the DELETE columns
drop_cols = [c for c in df_311.columns if "DELETE" in c]
df_311 = df_311.drop(columns = drop_cols)

# exclude any records with invalid Latitude/Longitude
df_311 = df_311[df_311['Latitude'] > 0]

# convert Opened/Closed to datetime
df_311['Opened'] = pandas.to_datetime(df_311['Opened'])

df_311['Closed'] = pandas.to_datetime(df_311['Closed'])

# subtract the Opened time from the Closed time to get the OpenTime duration
df_311['OpenTime'] = df_311['Closed'] - df_311['Opened']

In [2]:
df_311 = pandas.DataFrame.spatial.from_xy(
    df = df_311,
    x_column = 'Longitude', 
    y_column = 'Latitude',
    sr = 4326
)

In [ ]:
df_311[['Latitude','Longitude','SHAPE']].head()

In [4]:
df_cbg = pandas.DataFrame.spatial.from_featureclass(
    "./Tutorial_08.gdb/Census_Block_Groups"
)

In [ ]:
df_cbg.head()

In [6]:
df_join = df_311.spatial.join(
    right_df = df_cbg,
    how = 'left',
    op = 'intersects',
)

In [ ]:
df_join[pandas.isnull(df_join.geoid)]

In [ ]:
# create a GIS object
gis = arcgis.GIS()

# create a map and set our Area of Interest
qc_map = gis.map("San Francisco, CA")

# plot the census block groups
df_cbg.spatial.plot(
    colors="#fafafa",
    map_widget = qc_map,
)

# narrow down the 311 records to the one that didn't join
null_record = df_join[pandas.isnull(df_join.geoid)][['SHAPE','geoid']]



# plot the null record
null_record.spatial.plot(map_widget=qc_map)

qc_map

In [9]:
df_cbg_summary = df_join.groupby("geoid").agg(
    {
        "OpenTime": "mean",
        "CaseID": "count"
    }
)

In [ ]:
df_cbg_summary.dtypes

In [11]:
df_cbg_summary['OpenTime'] = df_cbg_summary['OpenTime'].dt.days

In [ ]:
df_cbg_summary.dtypes

In [13]:
df_cbg_summary = df_cbg.merge(
    df_cbg_summary, 
    how = 'left', 
    left_on = 'geoid', 
    right_on = 'geoid'
)

In [ ]:
df_cbg_summary.head()

In [ ]:
# create a new WebMap object
gis = arcgis.GIS()
cbg_map = gis.map("San Francisco, CA")

# exclude any null OpenTime values (in the middle of the bay)
df_to_map = df_cbg_summary[pandas.notna(df_cbg_summary.OpenTime)]

# plot the summary on the map
df_to_map.spatial.plot(map_widget=cbg_map)

# add a legend
cbg_map.legend.enabled = True

cbg_map

In [19]:
# get the renderer for the census block layer
renderer_manager = cbg_map.content.renderer(0)

# use the smart mapping capabilities to create a class breaks renderer
smart_mapper = renderer_manager.smart_mapping()

smart_mapper.class_breaks_renderer(
    break_type = 'color',
    field = 'OpenTime',
    classification_method = 'natural-breaks',
    num_classes = 5,
)

In [ ]:
df_cbg_summary.spatial.to_featureclass(
    "./Tutorial_08.gdb/OpenTime_311_Cases_by_CBG"
)

Copyright 2025 Esri